In [2]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os
from pathlib import Path

In [10]:
apps = pd.read_csv('../data/application_record.csv')
apps

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [12]:
schema = pa.Schema.from_pandas(apps)
schema

ID: int64
CODE_GENDER: string
FLAG_OWN_CAR: string
FLAG_OWN_REALTY: string
CNT_CHILDREN: int64
AMT_INCOME_TOTAL: double
NAME_INCOME_TYPE: string
NAME_EDUCATION_TYPE: string
NAME_FAMILY_STATUS: string
NAME_HOUSING_TYPE: string
DAYS_BIRTH: int64
DAYS_EMPLOYED: int64
FLAG_MOBIL: int64
FLAG_WORK_PHONE: int64
FLAG_PHONE: int64
FLAG_EMAIL: int64
OCCUPATION_TYPE: string
CNT_FAM_MEMBERS: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 2514

In [13]:
schema = schema.set(12, pa.field('FLAG_MOBIL', pa.bool_()))
schema = schema.set(13, pa.field('FLAG_WORK_PHONE', pa.bool_()))
schema = schema.set(14, pa.field('FLAG_PHONE', pa.bool_()))
schema = schema.set(15, pa.field('FLAG_EMAIL', pa.bool_()))
schema

ID: int64
CODE_GENDER: string
FLAG_OWN_CAR: string
FLAG_OWN_REALTY: string
CNT_CHILDREN: int64
AMT_INCOME_TOTAL: double
NAME_INCOME_TYPE: string
NAME_EDUCATION_TYPE: string
NAME_FAMILY_STATUS: string
NAME_HOUSING_TYPE: string
DAYS_BIRTH: int64
DAYS_EMPLOYED: int64
FLAG_MOBIL: bool
FLAG_WORK_PHONE: bool
FLAG_PHONE: bool
FLAG_EMAIL: bool
OCCUPATION_TYPE: string
CNT_FAM_MEMBERS: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 2514

In [15]:
apps.to_parquet('../data/apps.parquet', schema=schema)

In [16]:
apps.to_parquet('../data/partitioned', schema=schema, partition_cols=['NAME_INCOME_TYPE'])

In [17]:
pd.read_parquet('../data/partitioned', engine='pyarrow')

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,NAME_INCOME_TYPE
0,5008808,F,N,Y,0,270000.0,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,True,False,True,True,Sales staff,1.0,Commercial associate
1,5008809,F,N,Y,0,270000.0,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,True,False,True,True,Sales staff,1.0,Commercial associate
2,5008810,F,N,Y,0,270000.0,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,True,False,True,True,Sales staff,1.0,Commercial associate
3,5008811,F,N,Y,0,270000.0,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,True,False,True,True,Sales staff,1.0,Commercial associate
4,5008819,M,Y,Y,0,135000.0,Secondary / secondary special,Married,House / apartment,-17778,-1194,True,False,False,False,Laborers,2.0,Commercial associate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6837905,M,Y,Y,1,355050.0,Secondary / secondary special,Married,House / apartment,-15904,-2614,True,False,False,False,None,3.0,Working
438553,6837906,M,Y,Y,1,355050.0,Secondary / secondary special,Married,House / apartment,-15904,-2614,True,False,False,False,None,3.0,Working
438554,6839936,M,Y,Y,1,135000.0,Secondary / secondary special,Married,House / apartment,-12569,-2095,True,False,False,False,Laborers,3.0,Working
438555,6840222,F,N,N,0,103500.0,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,True,False,False,False,Laborers,1.0,Working


In [19]:
filters = [('NAME_INCOME_TYPE', 'in', ['Working', 'State servant'])]
pd.read_parquet('../data/partitioned/', filters=filters)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,NAME_INCOME_TYPE
0,5008868,F,N,Y,1,211500.0,Secondary / secondary special,Civil marriage,House / apartment,-16212,-7099,True,False,False,False,Core staff,3.0,State servant
1,5008870,F,N,Y,1,211500.0,Secondary / secondary special,Civil marriage,House / apartment,-16212,-7099,True,False,False,False,Core staff,3.0,State servant
2,5008981,M,Y,Y,0,112500.0,Secondary / secondary special,Married,House / apartment,-20874,-2381,True,False,True,False,Drivers,2.0,State servant
3,5008983,M,Y,Y,0,112500.0,Secondary / secondary special,Married,House / apartment,-20874,-2381,True,False,True,False,Drivers,2.0,State servant
4,5008984,M,Y,Y,0,112500.0,Secondary / secondary special,Married,House / apartment,-20874,-2381,True,False,True,False,Drivers,2.0,State servant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262285,6837905,M,Y,Y,1,355050.0,Secondary / secondary special,Married,House / apartment,-15904,-2614,True,False,False,False,None,3.0,Working
262286,6837906,M,Y,Y,1,355050.0,Secondary / secondary special,Married,House / apartment,-15904,-2614,True,False,False,False,None,3.0,Working
262287,6839936,M,Y,Y,1,135000.0,Secondary / secondary special,Married,House / apartment,-12569,-2095,True,False,False,False,Laborers,3.0,Working
262288,6840222,F,N,N,0,103500.0,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,True,False,False,False,Laborers,1.0,Working


In [23]:
file = '../data/partitioned/NAME_INCOME_TYPE=Working/3f9a745063b9417a9a58752ffbfa4f73-0.parquet'
metadata = pq.read_metadata(file)
metadata

  created_by: parquet-cpp-arrow version 13.0.0
  num_columns: 17
  num_rows: 226104
  num_row_groups: 14
  format_version: 2.6
  serialized_size: 30812

In [28]:
parquet_file = pq.ParquetFile(file)
ts = parquet_file.metadata.row_group(0)

In [30]:
beautiful_df = pd.DataFrame()
for nm in range(ts.num_columns):
    path_in_schema = ts.column(nm).path_in_schema
    compressed_size = ts.column(nm).total_compressed_size
    stats = ts.column(nm).statistics
    min_value = stats.min
    max_value = stats.max
    physical_type = stats.physical_type
    beautiful_df[path_in_schema] = pd.DataFrame([physical_type, min_value, max_value, compressed_size])
df = beautiful_df.T
df.columns = ['DTYPE', 'Min', 'Max', 'Compressed_Size_(KO)']
df

,DTYPE,Min,Max,Compressed_Size_(KO)
ID,INT64,5008804,6820978,94394
CODE_GENDER,BYTE_ARRAY,F,M,2252
FLAG_OWN_CAR,BYTE_ARRAY,N,Y,2173
FLAG_OWN_REALTY,BYTE_ARRAY,N,Y,1595
CNT_CHILDREN,INT64,0,14,3296
AMT_INCOME_TOTAL,DOUBLE,36000.0,900000.0,9422
NAME_EDUCATION_TYPE,BYTE_ARRAY,Academic degree,Secondary / secondary special,2806
NAME_FAMILY_STATUS,BYTE_ARRAY,Civil marriage,Widow,3378
NAME_HOUSING_TYPE,BYTE_ARRAY,Co-op apartment,With parents,1698
DAYS_BIRTH,INT64,-24376,-7980,25660


In [16]:
from collections import defaultdict
partitions = defaultdict(list)
for root, dirs, files in os.walk('../data/partitioned/'):
    if len(dirs) > 0:
        for i, partition in enumerate(dirs):
            key, part = partition.split('=')
            partitions[key].append(part)
            print(i, partition)
            
dict(partitions)
        


0 NAME_INCOME_TYPE=Student
1 NAME_INCOME_TYPE=Working
2 NAME_INCOME_TYPE=State%20servant
3 NAME_INCOME_TYPE=Commercial%20associate
4 NAME_INCOME_TYPE=Pensioner


{'NAME_INCOME_TYPE': ['Student',
  'Working',
  'State%20servant',
  'Commercial%20associate',
  'Pensioner']}

In [17]:
def get_all_partitions(path):
    partitions = {}
    i = 0
    for (_, partitions_layer, _) in os.walk(path):
        if len(partitions_layer)>0:
            key = partitions_layer[0].split('=')[0]
            partitions[key] = sorted([partitions_layer[i].split('=')[1] for i in range(len(partitions_layer))])
        # else:
        #     break
    return partitions

get_all_partitions('../data/partitioned/')

{'NAME_INCOME_TYPE': ['Commercial%20associate',
  'Pensioner',
  'State%20servant',
  'Student',
  'Working']}

In [21]:
def get_parquet_schema(path):
    for p in Path(path).rglob('*'):
        if p.is_file() and p.suffix == '.parquet':
            return pq.read_schema(p)
path = '../data/partitioned/'
schema = get_parquet_schema(path)
schema

ID: int64
CODE_GENDER: string
FLAG_OWN_CAR: string
FLAG_OWN_REALTY: string
CNT_CHILDREN: int64
AMT_INCOME_TOTAL: double
NAME_EDUCATION_TYPE: string
NAME_FAMILY_STATUS: string
NAME_HOUSING_TYPE: string
DAYS_BIRTH: int64
DAYS_EMPLOYED: int64
FLAG_MOBIL: bool
FLAG_WORK_PHONE: bool
FLAG_PHONE: bool
FLAG_EMAIL: bool
OCCUPATION_TYPE: string
CNT_FAM_MEMBERS: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [{"name": null, "field_n' + 2440